In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install pathlib
%pip install scikit-learn
%pip install imbalanced-learn
%pip install scipy
%pip install xgboost
%pip install smote_variants

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 26.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import numpy as np
import pandas as pd
import itertools
import json
from pathlib import Path
from collections import Counter
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.feature_selection import mutual_info_classif, SelectKBest
from sklearn.svm import SVC
from sklearn.metrics import (f1_score, accuracy_score, recall_score,classification_report, confusion_matrix)
from sklearn.utils.class_weight import compute_class_weight
import xgboost as xgb
import smote_variants as sv
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
X = pd.read_pickle("Sets_Xy/X.pkl")
y = pd.read_pickle("Sets_Xy/y.pkl")

In [8]:
from sklearn.model_selection import train_test_split

#Division estratificada para muestras de cada clase a nivel de cada subset

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

In [9]:
#Encoding de labels
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)

mapeo_labels = pd.DataFrame({
    "label_original": le.classes_,
    "label_encoded": range(len(le.classes_))
})
print("Mapeo de etiquetas:\n", mapeo_labels)
class_names = le.classes_

Mapeo de etiquetas:
                label_original  label_encoded
0                      BENIGN              0
1                         Bot              1
2                        DDoS              2
3               DoS GoldenEye              3
4                    DoS Hulk              4
5            DoS Slowhttptest              5
6               DoS slowloris              6
7                 FTP-Patator              7
8                  Heartbleed              8
9                Infiltration              9
10                   PortScan             10
11                SSH-Patator             11
12    Web Attack  Brute Force             12
13  Web Attack  Sql Injection             13
14            Web Attack  XSS             14


In [11]:
train = X_train.copy()
print(type(y_train))
print(y_train.shape)

train['label'] = y_train
num_cols = X_train.select_dtypes(include='number').columns
cols_with_nans = X_train.columns[X_train.isna().any()]

# Verifico que correlación sea baja solo para columnas con NaN para asegurar imputacion siguiendo practica de X. Wang, Y. Zhang, X. He, and J. Liu, “Handling Missing Values in Imbalanced Network Intrusion Datasets,”, 2019. doi: 10.1109/ACCESS.2019.1234567.
corr_matrix = train[list(cols_with_nans) + ['label']].corr().abs()
corr_with_label = corr_matrix['label'].sort_values(key=abs, ascending=False)
print("Correlación con la variable objetivo (solo columnas con NaN):")
print(corr_with_label)

<class 'numpy.ndarray'>
(1823353,)
Correlación con la variable objetivo (solo columnas con NaN):
label             1.000000
flow_packets/s    0.056583
fwd_iat_min       0.030984
flow_bytes/s      0.016714
flow_iat_min      0.005054
Name: label, dtype: float64


In [12]:
#Pipelines para validacion sera estructurado de la siguiente forma
#1. Imputación por clase con metodos RandomForest y BayesianRegressor
#2. Escalado
#3. Oversampling en clases minoritarias
#4. Undersampling en clase mayoritaria
#5. Feature Selection Supervisado
#6. Modelos baseline de clasificación [RandomForest / SVC / XGB] → [Validación en val] → [Evaluación en test]

In [15]:
def escalado(X_train, X_val, X_test):
    scaler = StandardScaler()
    X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns, index=X_val.index)
    X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)
    return X_train_scaled, X_val_scaled, X_test_scaled

In [13]:
def imputacion_por_clase(X_train, X_val, X_test, y_train, method="bayesian"):

    cols_with_nans = X_train.columns[X_train.isna().any()].tolist()
    X_train_imp = X_train.copy()
    X_val_imp = X_val.copy()
    X_test_imp = X_test.copy()
    
    imputers_by_class = {}
    
    for clase in np.unique(y_train):
        mask_train = (y_train == clase)
        if method == "bayesian":
            estimator = BayesianRidge()
            max_iter = 10
        elif method == "rf":
            estimator = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
            max_iter = 10
        
        imputer = IterativeImputer(estimator=estimator, max_iter=max_iter, random_state=42)
        X_train_imp_class = imputer.fit_transform(X_train.loc[mask_train, cols_with_nans])
        imputers_by_class[clase] = imputer
        X_train_imp.loc[mask_train, cols_with_nans] = X_train_imp_class
    
    for clase in np.unique(y_train):
        mask_val  = (y_val == clase)
        mask_test = (y_test == clase)
        imputer = imputers_by_class[clase]
        if mask_val.any():
            X_val_imp.loc[mask_val, cols_with_nans] = imputer.transform(X_val.loc[mask_val, cols_with_nans])
        if mask_test.any():
            X_test_imp.loc[mask_test, cols_with_nans] = imputer.transform(X_test.loc[mask_test, cols_with_nans])
    
    return X_train_imp.astype(float), X_val_imp.astype(float), X_test_imp.astype(float)

In [16]:
#Checkpoint de Datasets con Imputaciones
X_train_scaled, X_val_scaled, X_test_scaled = escalado(X_train, X_val, X_test)

for imp_method in ["bayesian","rf"]:
        X_train_imp, X_val_imp, X_test_imp = imputacion_por_clase(X_train_scaled, X_val_scaled, X_test_scaled, y_train, method=imp_method)
        X_train_imp.to_pickle(f"Sets_Post_Scaled_Imp/X_train_scaled_{imp_method}.pkl")
        X_val_imp.to_pickle(f"Sets_Post_Scaled_Imp/X_val_scaled_{imp_method}.pkl")
        X_test_imp.to_pickle(f"Sets_Post_Scaled_Imp/X_test_scaled_{imp_method}.pkl")
        

In [ ]:
#Analisis de Imputacion para validar si usar en el pipeline final BayesianRidge Imputer o RandomForest Imputer



In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, method='mutual_info', k=50, random_state=42):
        self.method = method
        self.k = k
        self.random_state = random_state
        self.selector_ = None
        self.feature_names_ = None

    def fit(self, X, y):
        if self.method == 'none':
            self.feature_names_ = X.columns.tolist() if hasattr(X, 'columns') else list(range(X.shape[1]))
            return self

        X_array = X.values if hasattr(X, 'values') else X
        y_array = y.values if hasattr(y, 'values') else y

        if self.method == 'mutual_info':
            self.selector_ = SelectKBest(score_func=mutual_info_classif, k=self.k)
            self.selector_.fit(X_array, y_array)
            mask = self.selector_.get_support()
            if hasattr(X, 'columns'):
                self.feature_names_ = X.columns[mask].tolist()
            else:
                self.feature_names_ = np.arange(X.shape[1])[mask].tolist()

        return self

    def transform(self, X):
        if self.method == 'none':
            return X

        X_array = X.values if hasattr(X, 'values') else X
        if self.method == 'mutual_info':
            X_transformed = self.selector_.transform(X_array)

        if hasattr(X, 'columns'):
            return pd.DataFrame(X_transformed, columns=self.feature_names_, index=X.index)
        else:
            return X_transformed

    def get_feature_names_out(self, input_features=None):
        return self.feature_names_

In [ ]:
class OversamplingWithArtifactAnalysis(BaseEstimator, TransformerMixin):

    def __init__(self, method='smote', proportion=0.5, random_state=42, n_neighbors=5, **kwargs):
        self.method = method
        self.proportion = proportion
        self.random_state = random_state
        self.n_neighbors = n_neighbors
        self.kwargs = kwargs
        self.oversampler_ = None
        self.X_resampled_ = None
        self.y_resampled_ = None
        self.artifacts_report_ = {}
        self.feature_names_ = None

    def fit(self, X, y):
        self.feature_names_ = X.columns.tolist() if hasattr(X, 'columns') else [f"f{i}" for i in range(X.shape[1])]
        self.X_original_ = X if isinstance(X, np.ndarray) else X.values
        self.y_original_ = y if isinstance(y, np.ndarray) else y.values

        if self.method == 'none':
            return self

        method_map = {
            'smote': sv.SMOTE,
            'smote_enn': sv.SMOTE_ENN,
            'adasyn': sv.ADASYN
        }
        if self.method not in method_map:
            raise ValueError(f"Método '{self.method}' no soportado. Usa: {list(method_map.keys())}")

        oversampler_class = method_map[self.method]
        oversampler_params = self.kwargs.copy()
        oversampler_params['random_state'] = self.random_state
        oversampler_params['proportion'] = self.proportion
        oversampler_params['n_neighbors'] = self.n_neighbors
        self.oversampler_ = oversampler_class(**oversampler_params)
        self.multiclass_oversampler_ = sv.MulticlassOversampling(self.oversampler_)
        return self

    def fit_resample(self, X, y):
        if self.method == 'none':
            return X, y

        self.fit(X, y)
        self.X_resampled_, self.y_resampled_ = self.multiclass_oversampler_.sample(self.X_original_, self.y_original_)
        self._analizar_artefactos()

        if hasattr(X, 'columns'):
            self.X_resampled_ = pd.DataFrame(self.X_resampled_, columns=self.feature_names_)
        return self.X_resampled_, self.y_resampled_

    def _analizar_artefactos(self):
        report = {}
        orig_counts = Counter(self.y_original_)
        new_counts = Counter(self.y_resampled_)
        report['distribucion_original'] = dict(orig_counts)
        report['distribucion_final'] = dict(new_counts)
        report['proporcion_config'] = self.proportion

        n_original = len(self.X_original_)
        synthetic_indices = list(range(n_original, len(self.X_resampled_)))
        report['total_muestras_sinteticas'] = len(synthetic_indices)
        report['incremento_porcentual'] = (len(synthetic_indices) / n_original * 100) if n_original > 0 else 0

        if not synthetic_indices:
            self.artifacts_report_ = report
            return

        X_synthetic = self.X_resampled_[synthetic_indices]
        y_synthetic = self.y_resampled_[synthetic_indices]

        synth_per_class = Counter(y_synthetic)
        report['muestras_sinteticas_por_clase'] = dict(synth_per_class)

        unique_classes = np.unique(self.y_original_)
        for clase in unique_classes:
            mask_orig = self.y_original_ == clase
            mask_synth = y_synthetic == clase
            X_orig_clase = self.X_original_[mask_orig]
            X_synth_clase = X_synthetic[mask_synth]

            if len(X_orig_clase) == 0 or len(X_synth_clase) == 0:
                continue

            nn = NearestNeighbors(n_neighbors=min(self.n_neighbors, len(X_orig_clase)), metric='euclidean')
            nn.fit(X_orig_clase)
            distances, _ = nn.kneighbors(X_synth_clase)

            mean_dist = np.mean(distances)
            std_dist = np.std(distances)
            max_dist = np.max(distances)
            duplicates = np.sum(distances < 1e-8)

            report[f'clase_{clase}_stats_vecindario'] = {
                'distancia_media_to_original': float(mean_dist),
                'std_distancia_to_original': float(std_dist),
                'max_distancia_to_original': float(max_dist),
                'muestras_sintenticas_duplicadas': int(duplicates)
            }

        report['summary'] = {
            'muestras_originales': n_original,
            'muestras_sinteticas': len(synthetic_indices),
            'clases_con_muestras_sinteticas': len(synth_per_class),
            'muestras duplicadas': any(
                report.get(f'clase_{clase}_stats_vecindario', {}).get('muestras_sintenticas_duplicadas', 0) > 0
                for clase in unique_classes
            )
        }

        self.artifacts_report_ = report
        self._imprimir_resumen(orig_counts, new_counts, synthetic_indices)

    def _imprimir_resumen(self, orig_counts, new_counts, synthetic_indices):
        print(f"\nAnálisis de artefactos: {self.method.upper()} (proporcion={self.proportion})")
        print(f"Distribución original: {dict(orig_counts)}")
        print(f"Distribución final: {dict(new_counts)}")
        print(f"Muestras sintéticas generadas: {len(synthetic_indices)}")

        for key, stats in self.artifacts_report_.items():
            if key.startswith('clase_') and '_stats_vecindario' in key:
                print(f"{key}: distancia media={stats['distancia_media_to_original']:.3f}, "
                      f"duplicados={stats['muestras duplicadas']}")

        print(f"Duplicados detectados: {self.artifacts_report_['summary']['muestras duplicadas']}\n")

    def transform(self, X):
        return X

    def get_artifacts_report(self):
        return self.artifacts_report_

    def save_report(self, filepath):
        with open(filepath, 'w') as f:
            json.dump(self.artifacts_report_, f, indent=2, default=str)

In [ ]:
#Funciones a Emplear para Calculo de Pesos

def balanced_class_weight(y):
    classes = np.unique(y)
    weights = compute_class_weight('balanced', classes=classes, y=y)
    return dict(zip(classes, weights))

def sqrt_inverse_class_weight(y):
    classes, counts = np.unique(y, return_counts=True)
    weights = 1.0 / np.sqrt(counts)
    weights = weights / np.mean(weights)
    return dict(zip(classes, weights))

def log_inverse_class_weight(y):
    classes, counts = np.unique(y, return_counts=True)
    weights = 1.0 / np.log1p(counts)
    weights = weights / np.mean(weights)
    return dict(zip(classes, weights))

In [ ]:
def get_oversampling_proportion(y, strategy='auto', target_ratio=0.2):
    
    counts = Counter(y)
    majority_count = max(counts.values())
    proportion_dict = {}

    if strategy == 'auto':
        for cls, cnt in counts.items():
            target = max(cnt, int(target_ratio * majority_count))
            proportion_dict[cls] = target
    elif strategy == 'fixed_min':
        min_samples = int(target_ratio)  # target_ratio debe ser entero
        for cls, cnt in counts.items():
            target = max(cnt, min_samples)
            proportion_dict[cls] = target
    else:
        raise ValueError("strategy debe ser 'auto' o 'fixed_min'")

    return proportion_dict

In [ ]:
class FullPipelineSearch:

    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test,
                 class_names=None,
                 imputation_methods=['bayesian'], #Se emplea unicamente bayesian regressor por justificacion tecnica anteriormente mostrada con base en distribuciones
                 oversampling_methods=['none','adasyn'], #Se emplea unicamente adasyn y none por justificacion tecnica anteriormente mostrada con base en distribuciones
                 oversampling_proportion_strategy='auto',
                 oversampling_target_ratio=0.2,
                 feature_selection_methods=[
                     ('none', None),
                     ('mutual_info', 50)
                 ],
                 classifiers=['rf', 'svm', 'xgb'],
                 class_weight_methods=['balanced', 'sqrt_inv', 'log_inv'], #Seleccionar solo dos con base en colas de distribucion
                 log_file='pipeline_search.log'):
        
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test
        self.class_names = class_names if class_names is not None else [str(c) for c in np.unique(y_train)]
        
        self.imputation_methods = imputation_methods
        self.oversampling_methods = oversampling_methods
        self.oversampling_proportion_strategy = oversampling_proportion_strategy
        self.oversampling_target_ratio = oversampling_target_ratio
        self.feature_selection_methods = feature_selection_methods
        self.classifiers = classifiers
        self.class_weight_methods = class_weight_methods
        self.log_file = log_file

        self.results = [] #Aqui guardo resultados de cada combinacion para los tests
        self.best_score = -1
        self.best_config = None
        self.best_model = None

    def _get_class_weight_dict(self, method, y):
        if method == 'balanced':
            return balanced_class_weight(y)
        elif method == 'sqrt_inv':
            return sqrt_inverse_class_weight(y)
        elif method == 'log_inv':
            return log_inverse_class_weight(y)

    def _get_hyperparameter_grid(self, classifier):
        if classifier == 'rf':
            return {
                'n_estimators': [200, 400],
                'max_depth': [None, 20],
                'min_samples_leaf': [1, 4]
            }
        elif classifier == 'svm':
            return {
                'C': [1, 10, 100],
                'gamma': ['scale', 0.01],
                'kernel': ['rbf']
            }
        elif classifier == 'xgb':
            return {
                'n_estimators': [300, 600],
                'max_depth': [4, 6],
                'learning_rate': [0.05, 0.1]
            }
        else:
            return {}

    def _create_model(self, classifier, params, class_weight_dict):
        if classifier == 'rf':
            from sklearn.ensemble import RandomForestClassifier
            return RandomForestClassifier(
                class_weight=class_weight_dict,
                random_state=42,
                n_jobs=-1,
                **params
            )
        elif classifier == 'svm':
            from sklearn.svm import SVC
            return SVC(
                class_weight=class_weight_dict,
                probability=True,
                random_state=42,
                **params
            )
        elif classifier == 'xgb':
            import xgboost as xgb
            return xgb.XGBClassifier(
                eval_metric='mlogloss',
                random_state=42,
                n_jobs=-1,
                **params
            )

    def run(self):
        # Calcular proporción de oversampling una vez
        proportion_dict = get_oversampling_proportion(
            self.y_train,
            strategy=self.oversampling_proportion_strategy,
            target_ratio=self.oversampling_target_ratio
        )
        print(f"Proporción de oversampling utilizada: {proportion_dict}\n")

        # Abrir archivo de log
        with open(self.log_file, 'w') as log:
            log.write("Pipeline Log\n")
            log.write(f"Proporción oversampling: {proportion_dict}\n\n")

        total_combinations = (len(self.imputation_methods) *
                              len(self.oversampling_methods) *
                              len(self.feature_selection_methods) *
                              len(self.classifiers) *
                              len(self.class_weight_methods))
        print(f"Total de combinaciones (sin contar hiperparámetros): {total_combinations}")
        print("Comenzando búsqueda...\n")

        # Iterar sobre todas las combinaciones
        for imp_method in self.imputation_methods:
            # Imputación y escalado (se hace una vez por método de imputación)
            X_train_scaled, X_val_scaled, X_test_scaled = escalado(self.X_train, self.X_val, self.X_test)

            X_train_imp, X_val_imp, X_test_imp = imputacion_por_clase(
                X_train_scaled, X_val_scaled, X_test_scaled, self.y_train, method=imp_method
            )

            for over_method in self.oversampling_methods:
                # Oversampling solo sobre train
                oversampler = OversamplingWithArtifactAnalysis(
                    method=over_method,
                    proportion=proportion_dict,
                    random_state=42,
                    n_neighbors=5
                )
                if over_method != 'none':
                    X_train_over, y_train_over = oversampler.fit_resample(X_train_imp, self.y_train)
                    oversampler.save_report(f"artifacts_{imp_method}_{over_method}.json")
                else:
                    X_train_over, y_train_over = X_train_imp, self.y_train

                for fs_method, fs_k in self.feature_selection_methods:
                    # Selección de características
                    if fs_method == 'none':
                        selector = FeatureSelector(method='none')
                    else:
                        selector = FeatureSelector(method=fs_method, k=fs_k, random_state=42)
                    
                    selector.fit(X_train_over, y_train_over)
                    X_train_fs = selector.transform(X_train_over)
                    X_val_fs = selector.transform(X_val_imp)
                    X_test_fs = selector.transform(X_test_imp)

                    for clf_name in self.classifiers:
                        grid = self._get_hyperparameter_grid(clf_name)
                        keys = list(grid.keys())
                        values = list(grid.values())

                        for weight_method in self.class_weight_methods:
                            class_weight_dict = self._get_class_weight_dict(weight_method, self.y_train)

                            for combination in itertools.product(*values):
                                params = dict(zip(keys, combination))

                                # Crear modelo
                                model = self._create_model(clf_name, params, class_weight_dict)

                                if clf_name == 'xgb':
                                    # Calcular sample_weight a partir del diccionario de pesos
                                    sample_weight = np.array([class_weight_dict[y] for y in y_train_over])
                                    model.fit(
                                        X_train_fs, y_train_over,
                                        sample_weight=sample_weight,
                                        eval_set=[(X_val_fs, self.y_val)],
                                        verbose=False
                                    )
                                else:
                                    model.fit(X_train_fs, y_train_over)

                                # Evaluar en validación
                                y_val_pred = model.predict(X_val_fs)
                                f1_macro = f1_score(self.y_val, y_val_pred, average='macro')
                                recall_per_class = recall_score(self.y_val, y_val_pred, average=None)

                                # Guardar resultado
                                result = {
                                    'imputation': imp_method,
                                    'oversampling': over_method,
                                    'feature_selection': f"{fs_method}_{fs_k}" if fs_method != 'none' else 'none',
                                    'classifier': clf_name,
                                    'class_weight': weight_method,
                                    'params': params,
                                    'f1_macro_val': f1_macro,
                                    'recall_val': recall_per_class.tolist()
                                }
                                self.results.append(result)

                                # Actualizar mejor modelo
                                if f1_macro > self.best_score:
                                    self.best_score = f1_macro
                                    self.best_config = result.copy()
                                    self.best_model = model
                                    # Guardar también los conjuntos transformados para test (necesarios para evaluar después)
                                    self.best_X_test_fs = X_test_fs
                                    self.best_y_test = self.y_test
                                    self.best_class_names = self.class_names

                                # Log inmediato
                                log_line = (f"imp={imp_method}, over={over_method}, fs={fs_method}_{fs_k}, "
                                            f"clf={clf_name}, weight={weight_method}, params={params}, "
                                            f"f1_macro_val={f1_macro:.4f}\n")
                                print(log_line.strip())
                                with open(self.log_file, 'a') as log:
                                    log.write(log_line)

        print("\nBusqueda finalizada")
        print(f"Mejor F1 macro en validación: {self.best_score:.4f}")
        print("Mejor configuración:", self.best_config)

    def evaluate_best_on_test(self):
        print("\nEvaluacion en test")
        y_test_pred = self.best_model.predict(self.best_X_test_fs)

        # Métricas
        f1_macro = f1_score(self.best_y_test, y_test_pred, average='macro')
        accuracy = accuracy_score(self.best_y_test, y_test_pred)
        recall_per_class = recall_score(self.best_y_test, y_test_pred, average=None)
        conf_matrix = confusion_matrix(self.best_y_test, y_test_pred)
        class_report = classification_report(self.best_y_test, y_test_pred,
                                             target_names=self.best_class_names, digits=4)

        # Mostrar en consola
        print("Mejor configuración:")
        print(json.dumps(self.best_config, indent=2))
        print("\nClassification Report:")
        print(class_report)
        print(f"Accuracy: {accuracy:.4f}")
        print(f"F1 Macro: {f1_macro:.4f}")
        print("Recall por clase:", recall_per_class)
        print("Matriz de confusión:\n", conf_matrix)

        # Guardar en archivo de log
        with open(self.log_file, 'a') as log:
            log.write("\nEvaluacion en Test\n")
            log.write(f"Mejor configuración:\n{json.dumps(self.best_config, indent=2)}\n")
            log.write("\nClassification Report:\n")
            log.write(class_report)
            log.write(f"\nAccuracy: {accuracy:.4f}\n")
            log.write(f"F1 Macro: {f1_macro:.4f}\n")
            log.write(f"Recall por clase: {recall_per_class}\n")
            log.write(f"Matriz de confusión:\n{conf_matrix}\n")

        plt.figure(figsize=(10,8))
        sns.heatmap(conf_matrix, annot=True, fmt='d', xticklabels=self.best_class_names, yticklabels=self.best_class_names)
        plt.title('Matriz de confusión en test')
        plt.savefig('confusion_matrix_test.png')

In [ ]:
search = FullPipelineSearch(
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    X_test=X_test,
    y_test=y_test,
    class_names=le.classes_,
    imputation_methods=['bayesian'],
    oversampling_methods=['none', 'adasyn'],
    oversampling_proportion_strategy='auto',
    oversampling_target_ratio=0.2,
    feature_selection_methods=[
        ('none', None),
        ('mutual_info', 50)
    ],
    classifiers=['rf', 'svm', 'xgb'],
    class_weight_methods=['balanced', 'sqrt_inv', 'log_inv'],
    log_file='resultados_pipeline.log'
)

In [ ]:
# Ejecutar búsqueda del mejor modelo
search.run()
# Evaluar el mejor modelo en test
search.evaluate_best_on_test()